<a href="https://colab.research.google.com/github/T-Sunm/Learn-pandas/blob/main/Exercise_Creating%2C_Reading_and_Writing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pitchfork-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F655%2F1252%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T141219Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D597744426b3e6298219d327137e38a1af5654c999f9579e473d71f34d5308a6771746507b0d622001f848ac80b73ffd4630c65a14c62b38820fe98d744be0f67b671046714c568a5365d0455a5c1c4edc5d65a2ec061be449da0e8786145d5a34594cbdbedd864f3758562672f2054ce68d9187e8f930bf88f92eaf60d49c895f760e9ffe48f708d7979b2d46d32524bacfdfe8bb2682dc29805f92aa9a2215e1718deaa1866003fe7833bb4c5850a1439ce8335d54d3440a030948a279900de70175230165781fd7e02c0d0aa69ce5944ece7239d5c5481e40c3ddd181efcd79c2cac8e28eddf1069cbd2c14e212ea9db66bf6aac100e11336d7db2b057dc6f,chess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2321%2F3919%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T141220Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db44c3bca76edac07f252594436ab0caae6b6ce927cc8018e26eb1265ff1e500c1de015390e1b5209b04f0d720bf0791af3ceebe3b6c013967d63474f89329332184b969f3032ffef5fecec00eebe18fba2fd1d3ee785623460b399b21a745eb3f2dcab9de9f60b3f297959e6e0c7be2d01a24f076fc5339d28ce3fd9040cb2bbd2e78b1465ce01d0dcae77bc9752e556e2fdc14e217e0de1cf413befb30dda5008c6a7f3b723af89439f1c6e9f7dcbbfa24f3f3ab00954481747f5fd1f073d5664917b166b637f4d781b94b86ad32c1031de1d4d7bdcc4df25e8452127ee01ba995a7acdaa6b34ab40929614838ffdbf2e85a3e8c171712f51630fdad83d140b,kepler-exoplanet-search-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2894%2F4877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T141220Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D276e9e5ff99e306e607b6957af1e22b417a45a1b9d89d80e67d410203f127a46d663d0f9c1897c2e8563fa7e05636fa170c9b5043ad2ff1a1be053bf398ed968db72d90981beb52c5b2be7cce3bf436bff3cee837b7c7465c020b37ef3fbb7deabac08759dbe2c40346b3eac64c1be203d586f011cee9a6482e34ad5ef88e9a07d5b2197891f350fe9be515be64fdb0cfdcfa0ad406e8696c54a4fbe31104373e99765b5803f339f9d1e77c7fedabf00da4924e06e7bc9d9daed0f1de80dac1bd558078be1f545d1c8acf5aee3e1d2309e4ceff77b5a42a76205d7cf1f8f3e239641114612c87cb6a3aa98c6bdb6212986115deda6b55eab846965abbdf36323,things-on-reddit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3491%2F5624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T141220Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3008946d3a1ef23b3356caa3d6e7d257a01393d7e6cc14109f80d2d247369ea51fc7335af8e2011a5f04113dfc0bae3ff0cf920ee96be29dd4641fda1c7ca5ff8bdbc1591684e841e5dfbaa7b8056806102d8990f671b307a1b3977b1e7c3171f4cfb78c05b4a31e95c0fff60058c88b912584c3859abf72f584e35e129347a0379a59b6c624a908e6f6e644d4bc889a93747a18350c74c361f2347b1511156a822025915817034f31e52038f0e612d118dfd7c59a4efab515290d57b7f2c0ea0ff7f3797b225f69b061f673b95575d1598995d07b1a37c32c6c3c9478263cb94145a6b5616621fc1f4be4ff616bd0c698021b01ada0ed8186eb01430c513627,wine-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1442%2F8172%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T141220Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2a0801d5e6dab6e4db0dd052188231e5411b45eb82bd099626d3809220d6fcf5d37d8b68ce5575757d4aefe1089adc4d706f21bb14fd1410f0941ba7cd7e19a7806cd099cdecac0f5c238f2e279369389e103609f00ff69573d4ddc327c952931e6278f4e3ca42cc3a63a7f60d8df6226d028cd739a161b589d6976ce5f29272202401290abdd061c1b0521756b3df7b7e7d4efc031ac9c9de1d341e1d40577c42a0c761fc4d247dc3dac31bd888c9c93d62a30435dde61aebf2e929bedf3312b949afa091eaf30fd60425635950c58d63d7eb707919ca0cfd8d897acddee3a97e089e7e162cbb37f64281b411ab140d80740eaa4c25e694cd5d184c84a9ba91,ramen-ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9366%2F13206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T141220Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dcd248c3c04fd4182a65d63ee890bc80556b86eb0d40066e8c6b58c861434e63abf48b1192ac76174c6d36577df153329f7e1ebc9c12b54d9200cf1976d8a128a2f0b26a184a3fb97c39abc7e21b2246196be881a6f0c1e68b5527de546cec696953a31c5f9089fd66b9c64e75a4b1becdf1cc1a380b9497206cefac5dd84208aac5b3c8c32e46a920a100cfcc8f679e0521aeb33b4d5d60e5770c775fd4da3b52a1ea6fada082111331f410963e7632d5c9cb5e4ced1aa449cd7050a36c05ec9152345656c035a23469cf14d173a6f451a8be24293c596a2981de624ae78c696db624aa8cdf755abf728e8ea8febf4cf2e1e6fcdf1088c93b623fc4ba684ac2a,powerlifting-database:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F179555%2F403916%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T141220Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1da4a88edae9e80af5a1d37d7beabc56c119448ee46397d33c6397fb4f39e7aeaf1fb4ad5cd4a823ed84f1c30cc6507aaffad2e90d954976eeb59b2d0075b01bc4a1ab7646da03f93fe2b39756567dbbc4e330e3a406a700506a68a6b570c0f4215c71436ad36cfec3913bb1448432aec5a3782fb7e2b485956df150b40bdd8ac901119d9e8b878dfc55ecc6537f00bf71201862a518b9f6750fb734116e1aa118c9f5c355f1575969192d076995f421247431d662604a7292eb80184a52022f5f9d655f42e07b6baba0a0a80689bb743e8987b067380e5b94abc9485b2bee17e02fff6a7ec3f33d7b7a0069d0467e590e0051db7d02752f3e8b3a9632aa1edc,youtube-new:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4549%2F466349%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T141220Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D597daa5de4d15506209a6ea36fd119c7bc14c52e4d37423e418be46603c95fadef41c60fd8df378c9b3ea2d2e3efd467616e5369a331e5bbea5066f5b52cbedc521b05da1d00cf79119fc7a409d1a0903b7b32aff7c1c26efb6cae1cb67dcd7a6fe57d79528916db25fba7f2ba33181055f15a29670ea533aa70eb7bf0794b66ecb14719e7dd8ba5db135074fbe91399d7b25aac428b16b48b63b489b2816eb07f599211885d0bc3ab2cb0dafd17e52236e4f554c9e1c86b8674d729bac2ff3d92f9b67dc5c2fb8e67af96f33fc4541cc2e1255e58390a9fd13cc932d214b6971ff869ee46a4627378e45c0c60bd70e3cd3aa79c63c980c6f8e7c26c94b7f47d,188-million-us-wildfires:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2478%2F1151655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T141220Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dbabdf11eb940f57fa08b5cbe3365d57a5ed6a8b260a55bf1ad8a5ebca112c985c56fab4538dbe175eb8dd8495b0667459b9fdb5c0dc55922613519fb095b48c7db7b5cd068e50e4589c91e07a9c27d926b781573146d7022b0c85ee4d56d76fc4a0838755222ec351f98de6a62b110729c690d52656fff51ff2caa133d78ce7a8f761862fd034452f52a37a15715b5bd9ef2f4364f1f2a876bfef84a56481f61b9e60075ca23aebed9965524e80fb4be190ce9504c63a42b4eaf2461b6e714c3c54b95cf1690e2fbc96f2061c89b2b179bdce994f4f230b5025c242dac3928b9284e1ddeee6af2eaa34886285a4598f3104b2244bd023dbfd8d99965de15ef8c,publicassistance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10128%2F5438389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T141220Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2fe87cb1864828e0285f2ab95cf2b8856d73a99dd0e018375f8cafdf8df9505b4dbff5333f72292a61f8da82366115e8570c6582c2f4c657904f4e8d9d9971d2ca9627ed5788a5c16366deee7a0db60a605175b8a4e2e35a39fb71929c9da10e64c134e4110872f0675ef4a18cfd34a79aab3c8d8c2a68e7b1fab5e9fad72b7dd3c2b1794fd5746f6b36d84fba3b9d6e58758048ada6d7022760569dcd79e6e82e703e89cf3a948d11b8058b59cfafef86ead9bcc5b5d45bb7728329d6320135fb9f4de0f30e540a572a33fa83acb96bbaf088d5b886f3c1b13ed12a82e367da523aaf97fb0b7f3c06b1e9bf8ebe8966870d6879923a53bbbd6ee9c40320ed7a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Pandas](https://www.kaggle.com/learn/pandas) course.  You can reference the tutorial at [this link](https://www.kaggle.com/residentmario/creating-reading-and-writing).**

---


# Introduction

The first step in most data analytics projects is reading the data file. In this exercise, you'll create Series and DataFrame objects, both by hand and by reading data files.

Run the code cell below to load libraries you will need (including code to check your answers).

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 5)
from learntools.core import binder; binder.bind(globals())
from learntools.pandas.creating_reading_and_writing import *
print("Setup complete.")

Setup complete.


# Exercises

## 1.

In the cell below, create a DataFrame `fruits` that looks like this:

![](https://storage.googleapis.com/kaggle-media/learn/images/Ax3pp2A.png)

In [ ]:
# Your code goes here. Create a dataframe matching the above diagram and assign it to the variable fruits.
fruits = pd.DataFrame({
    "Apples": [30],
    "Bananas": [21],
})

# Check your answer
q1.check()
fruits

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,Apples,Bananas
0,30,21


In [ ]:
#q1.hint()
#q1.solution()

## 2.

Create a dataframe `fruit_sales` that matches the diagram below:

![](https://storage.googleapis.com/kaggle-media/learn/images/CHPn7ZF.png)

In [ ]:
# Your code goes here. Create a dataframe matching the above diagram and assign it to the variable fruit_sales.
fruit_sales = pd.DataFrame({
    "Apples": [35, 41],
    "Bananas": [21, 34]},
    index = ['2017 Sales', '2018 Sales'])

# Check your answer
q2.check()
fruit_sales

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,Apples,Bananas
2017 Sales,35,21
2018 Sales,41,34


In [ ]:
#q2.hint()
#q2.solution()

## 3.

Create a variable `ingredients` with a Series that looks like:

```
Flour     4 cups
Milk       1 cup
Eggs     2 large
Spam       1 can
Name: Dinner, dtype: object
```

In [ ]:
ingredients = pd.Series(["4 cups", "1 cup", "2 large", "1 can"], index=["Flour", "Milk", "Eggs", "Spam"], name="Dinner")

# Check your answer
q3.check()
ingredients

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

Flour     4 cups
Milk       1 cup
Eggs     2 large
Spam       1 can
Name: Dinner, dtype: object

In [ ]:
#q3.hint()
#q3.solution()

## 4.

Read the following csv dataset of wine reviews into a DataFrame called `reviews`:

![](https://storage.googleapis.com/kaggle-media/learn/images/74RCZtU.png)

The filepath to the csv file is `../input/wine-reviews/winemag-data_first150k.csv`. The first few lines look like:

```
,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,"This tremendous 100% varietal wine[...]",Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and[...]",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,,Tinta de Toro,Bodega Carmen Rodríguez
```

In [ ]:
reviews = pd.read_csv("../input/wine-reviews/winemag-data_first150k.csv", index_col = 0)

# Check your answer
q4.check()
reviews

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
...,...,...,...,...,...,...,...,...,...,...
150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut Rosé,90,52.0,Champagne,Champagne,NaN,Champagne Blend,Gosset
150929,Italy,More Pinot Grigios should taste like this. A r...,NaN,90,15.0,Northeastern Italy,Alto Adige,NaN,Pinot Grigio,Alois Lageder


In [ ]:
#q4.hint()
#q4.solution()

## 5.

Run the cell below to create and display a DataFrame called `animals`:

In [ ]:
animals = pd.DataFrame({'Cows': [12, 20], 'Goats': [22, 19]}, index=['Year 1', 'Year 2'])
animals

,Cows,Goats
Year 1,12,22
Year 2,20,19


In the cell below, write code to save this DataFrame to disk as a csv file with the name `cows_and_goats.csv`.

In [ ]:
animals.to_csv('cows_and_goats.csv')

# Check your answer
q5.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
#q5.hint()
#q5.solution()

# Keep going

Move on to learn about **[indexing, selecting and assigning](https://www.kaggle.com/residentmario/indexing-selecting-assigning)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*